In [1]:
%load_ext autoreload
%autoreload 2

import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
import butterfly.album
import butterfly.Models
from itertools import combinations 
from joblib import parallel_backend
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score
import pickle
from sklearn.model_selection import GroupKFold
from random import sample
from scipy import stats
from sklearn.metrics import mean_absolute_error
import seaborn as sns

Using TensorFlow backend.
/home/mxenoc/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mxenoc/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mxenoc/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mxenoc/anaconda3/lib/python3.7/site-packages/tenso

In [2]:
#Define the predictor datasets
omics = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']

In [3]:
#Import your data
#DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [ ]:
pixels = 128 #image size
perplexity = 75 #perplexity for the tsne
scaler = "Standard"

albums_all_75 = []
for predictor_index in tqdm(range(len(omics))):
    
    #Get your predictor dataset
    predictors = list(omics)
    predictors.remove(omics[predictor_index])
    predictors = tuple(predictors)

    albums_all_75.append(butterfly.album.create_album(
        DF, predictors, pixels, perplexity, scaler))


 14%|█▍        | 1/7 [49:46<4:58:37, 2986.30s/it]

In [9]:
with open('albums_all_75.pkl', 'wb') as f:  
    pickle.dump(albums_all_75, f)

In [30]:
RF_predictor = []
for predictor_index in range(len(omics)):
    
    #Get your predictor dataset
    DFB = DF.copy()

    unwanted = DFB.columns[DFB.columns.str.startswith(omics[predictor_index])]
    DFB.drop(unwanted, axis=1, inplace=True)
    DFB.drop(['patient_TR', 'patientID', 'trimester'], axis=1, inplace=True)

    RF_predictor.append(DFB)

In [31]:
with open('RF_predictor.pkl', 'wb') as f:  
    pickle.dump(RF_predictor, f)

In [58]:
pixels = 40 #image size
perplexity = 25 #perplexity for the tsne

albums = Parallel(n_jobs=len(omics))(delayed(butterfly.album.create_album)
                                                    (DF, omics[al], pixels, perplexity)
                                                    for al in tqdm(range(len(omics))))



  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 31.98it/s]


In [59]:
with open('albums.pkl', 'wb') as f:  
    pickle.dump(albums, f)

In [35]:
n_samples = 30 #features to sample from each dataset

#Define the predictor datasets
omics = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']

#Import your data
#DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

responses = []
for predictor_index in range(len(omics)):
    
    #Get your response dataset
    DFB = DF.copy()
    response = sample([col for col in DFB if col.startswith(omics[predictor_index])], n_samples)
    responses.append(response)

In [36]:
with open('responses_30.pkl', 'wb') as f:  
    pickle.dump(responses, f)

In [ ]:
pic = albums_all_50[4][63]

In [ ]:
plot_im = plt.imshow(pic, cmap = 'RdGy')

In [ ]:
import seaborn as sns
sns.heatmap(pic)

In [32]:
print('test')

test
